In [1]:
from fastcore.all import *
import pandas as pd
import numpy as np
import datetime

In [2]:
def parse(file):
    df = pd.read_csv(file, usecols=[0,1,2], skiprows=1, 
                      header=None, parse_dates=True, index_col=0,
                      names=['time', 'temp', 'humidity']).resample('H').mean().reset_index()

    df['location'] = file.name.split('_export')[0]
    df['location'] = df.location.apply(lambda x: 'Backyard' if x =='Outside' else x)
    return df

In [3]:
def parse_files(path='./data/'):
    files = Path('./data/').ls(file_exts='.csv')
    return pd.concat([parse(f) for f in files]).set_index('time')

In [4]:
parse_files()

,temp,humidity,location
time,,,
2021-08-27 00:00:00,69.26,60.9,Backyard
2021-08-27 01:00:00,69.26,61.4,Backyard
2021-08-27 02:00:00,69.44,60.8,Backyard
2021-08-27 03:00:00,69.08,60.1,Backyard
2021-08-27 04:00:00,69.26,60.0,Backyard
...,...,...,...
2021-09-07 11:00:00,64.22,70.4,Lower Southeast
2021-09-07 12:00:00,64.22,70.2,Lower Southeast
2021-09-07 13:00:00,64.22,70.4,Lower Southeast


# Scrape Temp and Humidity Data

In [5]:
def subtract_day(strdate):
    "Subtract one day from strdate"
    return (pd.Timestamp(strdate) - pd.DateOffset(1)).strftime('%Y%m%d')

assert subtract_day('20210901') == '20210831'

In [6]:
def rmh(strdate):
    "remove hyphens from date"
    return pd.Timestamp(strdate).strftime('%Y%m%d')

assert rmh('2021-09-01') == '20210901'

In [7]:
def get_outside(api_key, startDate,endDate):
    startDate = subtract_day(startDate) # go back an additional date for a buffer
    url = f'https://api.weather.com/v1/location/KPDX:9:US/observations/historical.json?apiKey={api_key}&units=e&startDate={startDate}&endDate={endDate}'
    data = pd.DataFrame([{'time': x['valid_time_gmt'], 
                          'temp': x['temp'], 
                          'humidity': x['rh'],
                          'location': 'Outside'} for x in urljson(url)['observations']])
    data['time'] = data.time.apply(lambda x: pd.Timestamp(datetime.datetime.fromtimestamp(x)).round('60min'))
    data = data.set_index('time')
    return data

In [8]:
api_key='e1f10a1e78da46f5b10a1e78da96f525'
startDate='20210901'
endDate='20210907'

In [9]:
data = get_outside(api_key, startDate, endDate)
assert data.shape[1] == 3

In [30]:
data.loc['2021-09-05']

,temp,humidity,location
time,,,
2021-09-05 00:00:00,68,73,Outside
2021-09-05 01:00:00,67,76,Outside
2021-09-05 02:00:00,66,75,Outside
2021-09-05 03:00:00,64,78,Outside
2021-09-05 04:00:00,64,80,Outside
2021-09-05 05:00:00,63,84,Outside
2021-09-05 06:00:00,61,87,Outside
2021-09-05 07:00:00,62,84,Outside
2021-09-05 08:00:00,64,78,Outside


# Combine the data

In [17]:
def get_data(api_key, startDate, endDate):
    sensors = parse_files(path='./data/')
    print('finished sensors')
    outside = get_outside(api_key, rmh(startDate), rmh(endDate))
    
    return pd.concat([sensors.loc[startDate:endDate], outside.loc[startDate:endDate]], axis=0)

In [18]:
api_key='e1f10a1e78da46f5b10a1e78da96f525'
startDate='20210901'
endDate='20210907'

In [24]:
df = get_data(api_key, startDate='2021-09-01', endDate='2021-09-06')

finished sensors


In [25]:
df.groupby('location').count()

,temp,humidity
location,,
Backyard,144,144
Lower Northeast,144,144
Lower Southeast,144,144
Lower Southwest,144,144
Outside,144,144
Upper East (Garage),144,144
Upper West (Bedroom),144,144


In [27]:
df.to_csv('combined.csv', index=True)

In [40]:
df = pd.merge(df, df[df['location'] == 'Outside'], on='time', suffixes=('', '_outside'))

In [41]:
df.groupby('location').count()

,temp,humidity,temp_outside,humidity_outside,location_outside
location,,,,,
Backyard,144,144,144,144,144
Lower Northeast,144,144,144,144,144
Lower Southeast,144,144,144,144,144
Lower Southwest,144,144,144,144,144
Outside,144,144,144,144,144
Upper East (Garage),144,144,144,144,144
Upper West (Bedroom),144,144,144,144,144


In [42]:
df.to_csv('combined.csv', index=True)